In [1]:
%cd C:/Users/alapa/funiegan/

C:\Users\alapa\funiegan


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
import time
import numpy as np
from PIL import Image
from glob import glob
from ntpath import basename
from os.path import join, exists
# pytorch libs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.transforms as transforms

In [3]:
import argparse

for i in range(0,11):

    generator = f"checkpoints/FunieGAN/EUVP/generator_{i}.pth"
    print(generator) 
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", type=str, default=r"EUVP/test_samples/Inp")
    parser.add_argument("--sample_dir", type=str, default=r"EUVP/test_samples/result")
    parser.add_argument("--model_name", type=str, default="funiegan") # or "ugan"
    parser.add_argument("--model_path", type=str, default=generator)
    opt,unknown = parser.parse_known_args()

    print(opt.model_path)

    assert exists(opt.model_path), "model not found"
    os.makedirs(opt.sample_dir, exist_ok=True)
    is_cuda = torch.cuda.is_available()
    Tensor = torch.cuda.FloatTensor if is_cuda else torch.FloatTensor 

    if opt.model_name.lower()=='funiegan':
        from PyTorch.nets import funiegan
        model = funiegan.GeneratorFunieGAN()
    elif opt.model_name.lower()=='ugan':
        from nets.ugan import UGAN_Nets
        model = UGAN_Nets(base_model='pix2pix').netG
    else: 
        # other models
        pass

    model.load_state_dict(torch.load(opt.model_path))
    if is_cuda: model.cuda()
    model.eval()
    print ("Loaded model from %s" % (opt.model_path))

    ## data pipeline
    img_width, img_height, channels = 256, 256, 3
    transforms_ = [transforms.Resize((img_height, img_width), Image.BICUBIC),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
    transform = transforms.Compose(transforms_)

    dir_name = join(opt.sample_dir, f"generator_{i}")
    os.makedirs(dir_name, exist_ok=True)

    times = []
    test_files = sorted(glob(join(opt.data_dir, "*.*")))
    for path in test_files:
        inp_img = transform(Image.open(path))
        inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
        # generate enhanced image
        s = time.time()
        gen_img = model(inp_img)
        times.append(time.time()-s)
        # save output
        img_sample = torch.cat((inp_img.data, gen_img.data), -1)
        save_image(img_sample, join(dir_name, basename(path)), normalize=True)
        print ("Tested: %s" % path)

    if (len(times) > 1):
        print ("\nTotal samples: %d" % len(test_files)) 
        # accumulate frame processing times (without bootstrap)
        Ttime, Mtime = np.sum(times[1:]), np.mean(times[1:]) 
        print ("Time taken: %d sec at %0.3f fps" %(Ttime, 1./Mtime))
        print("Saved generated images in in %s\n" %(opt.sample_dir))

checkpoints/FunieGAN/EUVP/generator_0.pth
checkpoints/FunieGAN/EUVP/generator_0.pth


C:\Users\alapa\AppData\Local\Temp\ipykernel_7108\586843307.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(opt.model_path))


Loaded model from checkpoints/FunieGAN/EUVP/generator_0.pth
Tested: EUVP/test_samples/Inp\test_p0_.jpg
Tested: EUVP/test_samples/Inp\test_p100_.jpg
Tested: EUVP/test_samples/Inp\test_p101_.jpg
Tested: EUVP/test_samples/Inp\test_p102_.jpg
Tested: EUVP/test_samples/Inp\test_p104_.jpg
Tested: EUVP/test_samples/Inp\test_p105_.jpg
Tested: EUVP/test_samples/Inp\test_p106_.jpg
Tested: EUVP/test_samples/Inp\test_p107_.jpg
Tested: EUVP/test_samples/Inp\test_p109_.jpg
Tested: EUVP/test_samples/Inp\test_p110_.jpg
Tested: EUVP/test_samples/Inp\test_p112_.jpg
Tested: EUVP/test_samples/Inp\test_p113_.jpg
Tested: EUVP/test_samples/Inp\test_p114_.jpg
Tested: EUVP/test_samples/Inp\test_p115_.jpg
Tested: EUVP/test_samples/Inp\test_p116_.jpg
Tested: EUVP/test_samples/Inp\test_p117_.jpg
Tested: EUVP/test_samples/Inp\test_p118_.jpg
Tested: EUVP/test_samples/Inp\test_p119_.jpg
Tested: EUVP/test_samples/Inp\test_p11_.jpg
Tested: EUVP/test_samples/Inp\test_p120_.jpg
Tested: EUVP/test_samples/Inp\test_p121_.jp